In [ ]:
import os
from PIL import Image, ImageEnhance
import numpy as np

def apply_mask(image, mask):
    """
    将掩码应用到图片上，保留掩码区域内的图片部分
    """
    image_np = np.array(image)
    mask_np = np.array(mask)
    
    # 增加掩码对比度
    enhancer = ImageEnhance.Contrast(mask)
    mask = enhancer.enhance(2.0)
    
    mask_np = np.array(mask)
    
    # 将掩码二值化，保留白色部分（255）
    mask_binary = (mask_np > 128).astype(np.uint8)  # 使用阈值化处理掩码
    
    # 应用掩码，保留白色区域
    masked_image = image_np * mask_binary[:, :, np.newaxis]
    
    # 确保掩码区域透明处理
    alpha_channel = mask_binary * 255
    masked_image = np.dstack((masked_image, alpha_channel))
    
    return Image.fromarray(masked_image.astype(np.uint8))

def merge_images(image_dir, mask_dir, output_dir):
    """
    合并图片和掩码，并保存到新的文件夹
    """
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    
    for i in range(1, 1001):  # 假设图片编号从1到1000
        image_path = os.path.join(image_dir, f'{i}.jpg')
        mask_path = os.path.join(mask_dir, f'{i}.png')
        
        if not os.path.exists(image_path) or not os.path.exists(mask_path):
            print(f"图片或掩码缺失: {image_path}, {mask_path}")
            continue
        
        image = Image.open(image_path).convert('RGB')
        mask = Image.open(mask_path).convert('L')  # 假设掩码是灰度图
        
        masked_image = apply_mask(image, mask)
        output_path = os.path.join(output_dir, f'{i}.png')
        masked_image.save(output_path, 'PNG')  # 保存为PNG格式以保留透明度
        print(f"图片已合并并保存到 {output_path}！")

# 定义图片、掩码和输出文件夹路径
image_dir = r'/data1/dxw_data/llm/redbook-ture/contentVM-all-img'  # 替换为你的正常图片文件夹路径
mask_dir = r'/data1/dxw_data/llm/redbook-ture/contentVM-all-imgseg'  # 替换为你的掩码图片文件夹路径
output_dir = r'/data1/dxw_data/llm/redbook-ture/combine_final'  # 替换为你的输出文件夹路径

merge_images(image_dir, mask_dir, output_dir)
